In [1]:
import matplotlib.pyplot as plt
import numpy as np              
import pandas as pd          
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, precision_score, recall_score,\
    f1_score, accuracy_score, classification_report
#import keras_tuner as kt
import os
import tensorflow as tf
from FLPyfhelin import *
from Pyfhel import Pyfhel, PyPtxt, PyCtxt
import time

train_path = 'image/Train'
test_path = 'image/Test'
#folder = 'image'
batch_size = 32
#num_client =2
SCALE=1
epoch=10
input_size  = (int(256*SCALE), int(256*SCALE), 3)
image_size  = (int(256*SCALE), int(256*SCALE))


In [3]:
#Generate and export public key, just need to run it once
HE=gen_pk(s=128, m=1024)
keys ={}
keys['HE'] = HE
keys['con'] = HE.to_bytes_context()
keys['pk'] = HE.to_bytes_publicKey()
keys['sk'] = HE.to_bytes_secretKey()
    
filename =  "privatekey.pickle"
with open(filename, 'wb') as handle:
    pickle.dump(keys, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
HE

<Pyfhel obj at 0x1c7366e1f10, [pk:Y, sk:Y, rtk:-, rlk:-, contx(p=65537, m=1024, base=2, sec=128, dig=64i.32f, batch=False)]>

In [2]:
filename =  "privatekey.pickle"
with open(filename, 'rb') as handle:
        key = pickle.load(handle)

HE = key['HE']
HE.from_bytes_context(key['con'])
HE.from_bytes_publicKey(key['pk'])
HE.from_bytes_secretKey(key['sk'])
HE

NameError: name 'pickle' is not defined

In [10]:
num_of_client_list = [2]
prec_list = []
recall_list = []
f1_list = []
acc_list = []
t =[]

for j, num_client in enumerate(num_of_client_list):
        print('Number of clients: ',num_client)
        start = time.time()

         
        print('Prepare dataset')
        df_train = prep_df(train_path,shuffle=True)
        df_test = prep_df(test_path,shuffle=False)
        test_ds = get_test_data(df_test,train_path)
        
        print('Build main model')
        model = create_model()
        model.save('main_model.hdf5')

        print("Train Clients")
        train_clients(df_train, train_path, num_client,epoch)
                
        print("Export client weights")
        export_encrypted_clients_weights(num_client)
                
        print("Aggregate Weights")
        main_model_dict = aggregate_encrypted_weights(num_client)
        filename="weights/aggregated.pickle"
        print("Export Aggregate Weights")
        export_weights(filename ,main_model_dict)
                
        agg_model = decrypt_import_weights(filename)

        print("Run predictions on aggregated model")
        preds = agg_model.predict(test_ds,verbose=1)
        predictions = preds.copy()
        predictions = [np.argmax(x) for x in predictions]
        
        print("Run prepare confusion matrix")
        prec_list.append(precision_score(test_ds.classes, predictions,average='weighted'))
        recall_list.append(recall_score(test_ds.classes, predictions,average='weighted'))
        f1_list.append(f1_score(test_ds.classes, predictions,average='weighted'))
        acc_list.append(accuracy_score(test_ds.classes, predictions))
        end = time.time()
        t.append(end-start)
        
        

Number of clients:  2
Prepare dataset
Found 400 validated image filenames belonging to 2 classes.
Build main model
Train Clients
Found 720 validated image filenames belonging to 2 classes.
Found 80 validated image filenames belonging to 2 classes.
Epoch 1/10
23/23 [==============================] - ETA: 0s - loss: 0.6754 - accuracy: 0.5458 
Epoch 00001: accuracy improved from -inf to 0.54583, saving model to weights\client_1.ckpt
23/23 [==============================] - 540s 23s/step - loss: 0.6754 - accuracy: 0.5458 - val_loss: 0.5827 - val_accuracy: 0.7125 - lr: 0.0010
Epoch 2/10
23/23 [==============================] - ETA: 0s - loss: 0.3930 - accuracy: 0.8486 
Epoch 00002: accuracy improved from 0.54583 to 0.84861, saving model to weights\client_1.ckpt
23/23 [==============================] - 665s 29s/step - loss: 0.3930 - accuracy: 0.8486 - val_loss: 0.1553 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 3/10
23/23 [==============================] - ETA: 0s - loss: 0.1869 - accuracy: 0.

In [11]:
sum_list =[prec_list,recall_list,f1_list,acc_list]
client = num_of_client_list
idx=['Precision','Recall','F1 Score', 'Accuracy']

df = pd.DataFrame(sum_list, index=idx, columns=client)
df


,2
Precision,0.872779
Recall,0.842500
F1 Score,0.839235
Accuracy,0.842500


In [12]:
time_list =[t]
client = num_of_client_list
idx=['Time']

df = pd.DataFrame(time_list, index=idx, columns=client)
df

,2
Time,6583.635993


In [ ]:
import pickle
model=load_weights('1')
encrypted_weights={}

for i in range(len(model.layers)):
        if model.layers[i].get_weights()!=[]:
            encrypted =[]
            weights = model.layers[i].get_weights()   

            for j in range(len(weights)):
                    array= weights[j]
                    encrypted_weights['c_'+str(i)+'_'+str(j)]=array

filename =  "plainweights.pickle"
export_weights(filename, encrypted_weights)